In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

In [2]:
#Load in old dataframe 
df = pd.read_csv('Water_FINAL.csv', index_col = 0, low_memory = False)

C:\Users\jvhua\Miniconda3\envs\tensflow\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#Drop unnecessary columns 
df = df.drop(columns = 'Unnamed: 0.1')

In [4]:
df.columns

Index(['Agency', 'BiasPC', 'CloudCover', 'Comments', 'Cruise', 'Database',
       'Date', 'FieldActivityEventType', 'FieldActivityRemark', 'FlowStage',
       'GaugeHeight', 'GroupCode', 'HUC12', 'Lab', 'Latitude', 'Layer',
       'Longitude', 'LowerPycnocline', 'MeasureValue', 'Method',
       'ModifiedDate', 'Other Comments', 'Other Conditions', 'Parameter',
       'ParameterName_CBP', 'ParameterName_CMC', 'PrecipType', 'PrecisionPC',
       'Pressure', 'Problem', 'Program', 'Project', 'Qualifier', 'Rainfall',
       'Rainfall Within 24 Hours', 'Rainfall Within 48 Hours', 'SampleDepth',
       'SampleId', 'SampleReplicateType', 'SampleType', 'Sea State', 'Source',
       'Station', 'StationCode', 'StationName', 'Stream Flow', 'Tidal Stage',
       'TideStage', 'TierLevel', 'Time', 'TotalDepth', 'Unit',
       'UpperPycnocline', 'Water Color', 'Water Color Description',
       'Water Odor', 'Water Odor Description', 'Water Surfaces', 'WaveHeight',
       'Weather Conditions Day Before

In [5]:
#Take interested columns 
df2 = df.loc[:, ['Latitude', 
       'Longitude', 'Date', 'Time', 'HUCNAME_', 'FIPS_', 'Point', 'Parameter',
       'ParameterName_CBP', 'ParameterName_CMC', 'MeasureValue', 'Unit', 'SampleId']]

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2708717 entries, 0 to 2708716
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Latitude           float64
 1   Longitude          float64
 2   Date               object 
 3   Time               object 
 4   HUCNAME_           object 
 5   FIPS_              int64  
 6   Point              object 
 7   Parameter          object 
 8   ParameterName_CBP  object 
 9   ParameterName_CMC  object 
 10  MeasureValue       float64
 11  Unit               object 
 12  SampleId           float64
dtypes: float64(4), int64(1), object(8)
memory usage: 289.3+ MB


In [7]:
df2['Parameter'].unique()

array(['CHL.3', 'DO.6', 'NO3N.5', 'SA.9', 'DO.5', 'NO3N.6', 'WC.2',
       'WT.8', 'PH.6', 'CO.8', 'AT.3', 'OP.1', 'WT.1', 'NH4N.1', 'WT.4',
       'AT.11', 'AT.2', 'CHL.1', 'ECOLI.4', 'TP.3', 'NO3N.1', 'ECOLI.6',
       'CO.9', 'WC.7', 'OP.3', 'ECOLI.1', 'TDS.1', 'WT.5', 'WC.5', 'WC.6',
       'AT.1', 'AT.10', 'NO3N.2', 'TDS.3', 'AT.9', 'ENT.2', 'WT.10',
       'TD.1', 'WC.9', 'SA.3', 'SA.6', 'PH.11', 'DO.3', 'NO3N.4', 'DO.4',
       'SA.2', 'TSS.1', 'CO.1', 'PH.10', 'OP.2', 'WC.12', 'CO.4', 'WT.3',
       'WC.1', 'CO.2', 'ALKY.6', 'SA.10', 'PH.8', 'OP.8', 'NO2NO3.1',
       'DO.2', 'DO.7', 'WC.4', 'DO.15', 'TSS.3', 'OP.7', 'WC.8', 'CHL.2',
       'WT.12', 'TN.2', 'CO.5', 'OP.4', 'AT.4', 'TDS.2', 'NO3N.3', 'WT.2',
       'WT.6', 'PH.9', 'SA.8', 'CHL.4', 'WT.9', 'SA.1', 'PH.7', 'PH.1',
       'DO.9', 'TKN.1', 'ENT.1', 'WT.7', 'DO.14', 'AT.5', 'OP.6', 'TP.2',
       'PH.5', 'NO2NO3.4', 'TP.1', 'TN.1', 'PH.4', 'PH.2', 'ECOLI.2',
       'DO.1', 'PH.3', 'AT.6', 'DO.8', 'TN.3', 'WT.13', 'AL

In [8]:
#lower case measurement unit as it some were uppercased for some odd reason 
df2['Unit'] = df2['Unit'].apply(lambda x: x.lower())

In [9]:
#combine the parameter and unit, the reason why I chose CBP is because it's more complete choosing parameter you only have chemical name 
#the problem with that and using only the first two letters is that they have different measurements in the rows 
df2['Parameter and Unit'] = df2['ParameterName_CBP'] + ' '+ df2['Unit']

In [10]:
#Pivot the table 
df_pivot = df2.pivot_table('MeasureValue', ['Date', 'SampleId', 'Longitude', 'Latitude', 'Time'], 'Parameter and Unit')

In [11]:
df_pivot = df_pivot.reset_index()

In [12]:
#df3.groupby(['Date', 'SampleId'])['MeasureValue'].aggregate('mean').unstack().reset_index()

In [13]:
#Not enough observations to keep and not sure what it is 
columns = ['Not in CBP Database cfu', 'Not in CBP Database mg/l',
       'Not in CBP Database mpn', 'Not in CBP Database ntu',
       'Not in CBP Database ppb', 'Not in CBP Database ppt',
       'Not in CBP Database su', 'Not in CBP Database ug/l']

df_pivot = df_pivot.drop(columns = columns)

In [14]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132449 entries, 0 to 132448
Data columns (total 33 columns):
 #   Column                                                                 Non-Null Count   Dtype  
---  ------                                                                 --------------   -----  
 0   Date                                                                   132449 non-null  object 
 1   SampleId                                                               132449 non-null  float64
 2   Longitude                                                              132449 non-null  float64
 3   Latitude                                                               132449 non-null  float64
 4   Time                                                                   132449 non-null  object 
 5   ACTIVE CHLOROPHYLL-A ug/l                                              58164 non-null   float64
 6   AIR TEMPERATURE DEG deg c                                              16670

A Secchi disk is an 8-inch (20 cm) disk with alternating black and white quadrants. It is lowered into the water of a lake until it can no longer be seen by the observer. This depth of disappearance, called the Secchi depth, is a measure of the transparency of the water.

Turbidity is a measurement of the amount of suspended material in the water. It describes the clarity of water. Suspended materials in water, such as clay, silt, and algae, reduce water clarity and cause turbidity.

Parameter Importance
Turbidity is an important indicator of the amount of suspended sediment in water, which can have many negative effects on aquatic life. The suspended sediments that cause turbidity can block light to aquatic plants, smother aquatic organisms, and carry contaminants and pathogens, such as lead, mercury, and bacteria.

Long-term turbidity monitoring is very important because it helps us measure the amount of soil erosion due to construction and agriculture, and helps to determine the impact of our land-use practices and increasing urban development on water quality.

How it is Measured
Measuring turbidity is achieved by evaluating the amount of light scattered in water. This can be done with simple visual assessments, such as the original Jackson Candle turbidimeter, or more accurate, technological methods, such as the nephelometric method. Turbidity is reported in either NTUs or JTUs. These units refer to the method used to measure turbidity.

The original method to measure turbidity was the Jackson candle Turbidimeter, which was devised around the turn of the century. The Jackson Candle Turbidimeter consists of flat-bottomed glass tube that sits over a candle. A water sample is poured into the tube until the visual image of the candle flame diffuses into a uniform glow. The depth of the sample corresponds to a certain number of Jackson turbidity units, or JTUs.

The most accurate way to measure turbidity is with the nephelometric method. This method shines a concentrated beam of light through a water sample, and then measures the amount of scattered light at a 90 degree angle to the light source. The amount of scattered light is then reported in NTUs. The nephelometric method reflects a more precise measurement and is used by government agencies and in scientific research.

Definition for Total Nitrogen
The NPI NEPM states that total nitrogen refers only to those amounts of nitrogen
that give rise to nitrate/nitrite ions. Since this may be possible for a range of
nitrogen compounds in a range of environmental conditions, the following
pragmatic definition can be applied:
Total Nitrogen is the sum of nitrate (NO3), nitrite (NO2), organic nitrogen and
ammonia (all expressed as N). Note that for laboratory analysis purposes, Total
Kjeldahl Nitrogen (TKN) is a test performed that is made up of both organic
nitrogen and ammonia.

In [15]:
#drop columns with a repeated measures that aren't as accurate as the ones kept 
df_pivot2 = df_pivot.drop(columns = ['DO SATURATION USING PROBE UNITS IN PERCENT pct', 'ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE) ppm',
                         'SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU) mg/l', 
                          'SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU) pps', 'SPECIFIC CONDUCTIVITY ms/cm',
                         'TURBIDITY; JACKSON CANDLE METHOD (FORWARD SCATTER) jtu', 'TURBIDITY; NEPHELOMETRIC METHOD fau'])

In [16]:
df_pivot2.describe().T

,count,mean,std,min,25%,50%,75%,max
Parameter and Unit,,,,,,,,
SampleId,132449.0,162353.172225,189763.995712,1.000000,1.000000,62677.000000,406157.000000,462091.000000
Longitude,132449.0,-76.897939,0.848614,-80.437078,-77.339640,-76.713330,-76.294670,-74.835244
Latitude,132449.0,39.016295,1.329020,36.740040,38.044300,38.909378,39.670480,42.806720
ACTIVE CHLOROPHYLL-A ug/l,58164.0,13.605103,21.608400,0.000000,3.738000,7.830000,15.220000,1262.909975
AIR TEMPERATURE DEG deg c,16670.0,17.425392,8.903081,-14.000000,10.700000,18.777778,24.500000,83.000000
AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l,64686.0,0.046587,0.073753,-0.232000,0.011000,0.025500,0.054700,2.553000
DISSOLVED OXYGEN IN MG/L mg/l,113018.0,8.870668,2.820095,0.000000,6.984000,8.800000,10.800000,175.000000
E.COLI BACTERIA mpn,2612.0,295.614460,863.892346,0.000000,20.000000,70.000000,260.000000,27487.500000
NITRATE NITROGEN AS N (FILTERED SAMPLE) mg/l,59948.0,1.080424,1.983920,-0.012167,0.024269,0.330950,1.228608,20.000000


In [17]:
df_pivot2

Parameter and Unit,Date,SampleId,Longitude,Latitude,Time,ACTIVE CHLOROPHYLL-A ug/l,AIR TEMPERATURE DEG deg c,AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l,DISSOLVED OXYGEN IN MG/L mg/l,E.COLI BACTERIA mpn,...,TOTAL ALKALINITY AS CACO3 mg/l,TOTAL DEPTH m,TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C mg/l,TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE) mg/l,TOTAL NITROGEN mg/l,TOTAL PHOSPHORUS mg/l,TOTAL SUSPENDED SOLIDS mg/l,TURBIDITY TUBE cm,TURBIDITY; NEPHELOMETRIC METHOD ntu,WATER TEMPERATURE DEG deg c
0,01/01/1993,1.0,-77.18692,40.17999,12:00:00,NaN,NaN,NaN,11.60,NaN,...,NaN,NaN,290.0,NaN,NaN,NaN,NaN,NaN,2.40,9.25
1,01/01/1993,1.0,-77.18606,40.18844,12:00:00,NaN,NaN,NaN,11.00,NaN,...,NaN,NaN,385.0,NaN,NaN,NaN,NaN,NaN,4.15,8.50
2,01/01/1993,1.0,-77.18537,40.17663,12:00:00,NaN,NaN,NaN,11.15,NaN,...,NaN,NaN,385.0,NaN,NaN,NaN,NaN,NaN,0.75,9.50
3,01/01/1993,1.0,-77.18507,40.17667,12:00:00,NaN,NaN,NaN,11.25,NaN,...,NaN,NaN,330.0,NaN,NaN,NaN,NaN,NaN,2.00,8.85
4,01/01/1993,1.0,-77.18423,40.19528,12:00:00,NaN,NaN,NaN,10.70,NaN,...,NaN,NaN,395.0,NaN,NaN,NaN,NaN,NaN,4.15,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132444,9/9/2019,459510.0,-76.12810,39.69030,08:30:00,NaN,NaN,0.046,8.83,NaN,...,49.6,NaN,188.0,0.705,7.198,0.049,5.0,NaN,NaN,20.86
132445,9/9/2019,459586.0,-76.36774,39.94649,11:00:00,NaN,NaN,0.051,9.22,NaN,...,193.0,NaN,406.0,NaN,NaN,0.158,6.0,NaN,NaN,20.58
132446,9/9/2019,461212.0,-77.93889,39.42361,13:10:00,NaN,NaN,0.010,9.90,NaN,...,NaN,NaN,NaN,NaN,NaN,0.020,15.0,NaN,NaN,20.90
132447,9/9/2019,461228.0,-77.83183,39.48308,09:10:00,NaN,NaN,0.020,9.80,NaN,...,NaN,NaN,NaN,NaN,NaN,0.010,15.0,NaN,NaN,15.20


In [18]:
#combine time to merge with Bryans dataset 
df_pivot2['DateTime'] = df_pivot2['Date'] + ' ' + df_pivot2['Time']

In [19]:
#change to datetime 
import dateutil.parser
df_pivot2['DateTime'] = df_pivot2['DateTime'].apply(lambda x :  dateutil.parser.parse(x))

In [20]:
df_pivot2 = df_pivot2.drop(columns = ['Date', 'Time'])

In [21]:
df_pivot2

Parameter and Unit,SampleId,Longitude,Latitude,ACTIVE CHLOROPHYLL-A ug/l,AIR TEMPERATURE DEG deg c,AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l,DISSOLVED OXYGEN IN MG/L mg/l,E.COLI BACTERIA mpn,NITRATE NITROGEN AS N (FILTERED SAMPLE) mg/l,ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE) mg/l,...,TOTAL DEPTH m,TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C mg/l,TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE) mg/l,TOTAL NITROGEN mg/l,TOTAL PHOSPHORUS mg/l,TOTAL SUSPENDED SOLIDS mg/l,TURBIDITY TUBE cm,TURBIDITY; NEPHELOMETRIC METHOD ntu,WATER TEMPERATURE DEG deg c,DateTime
0,1.0,-77.18692,40.17999,NaN,NaN,NaN,11.60,NaN,6.200,0.150,...,NaN,290.0,NaN,NaN,NaN,NaN,NaN,2.40,9.25,1993-01-01 12:00:00
1,1.0,-77.18606,40.18844,NaN,NaN,NaN,11.00,NaN,6.200,0.100,...,NaN,385.0,NaN,NaN,NaN,NaN,NaN,4.15,8.50,1993-01-01 12:00:00
2,1.0,-77.18537,40.17663,NaN,NaN,NaN,11.15,NaN,5.300,0.100,...,NaN,385.0,NaN,NaN,NaN,NaN,NaN,0.75,9.50,1993-01-01 12:00:00
3,1.0,-77.18507,40.17667,NaN,NaN,NaN,11.25,NaN,5.900,0.100,...,NaN,330.0,NaN,NaN,NaN,NaN,NaN,2.00,8.85,1993-01-01 12:00:00
4,1.0,-77.18423,40.19528,NaN,NaN,NaN,10.70,NaN,6.050,0.150,...,NaN,395.0,NaN,NaN,NaN,NaN,NaN,4.15,8.50,1993-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132444,459510.0,-76.12810,39.69030,NaN,NaN,0.046,8.83,NaN,NaN,0.025,...,NaN,188.0,0.705,7.198,0.049,5.0,NaN,NaN,20.86,2019-09-09 08:30:00
132445,459586.0,-76.36774,39.94649,NaN,NaN,0.051,9.22,NaN,NaN,0.123,...,NaN,406.0,NaN,NaN,0.158,6.0,NaN,NaN,20.58,2019-09-09 11:00:00
132446,461212.0,-77.93889,39.42361,NaN,NaN,0.010,9.90,NaN,1.695,0.011,...,NaN,NaN,NaN,NaN,0.020,15.0,NaN,NaN,20.90,2019-09-09 13:10:00
132447,461228.0,-77.83183,39.48308,NaN,NaN,0.020,9.80,NaN,2.460,0.004,...,NaN,NaN,NaN,NaN,0.010,15.0,NaN,NaN,15.20,2019-09-09 09:10:00


In [22]:
df_pivot3 = df_pivot2[df_pivot2['DateTime'] > '2009-12-31']

In [23]:
df_pivot3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96603 entries, 45 to 132448
Data columns (total 25 columns):
 #   Column                                                                Non-Null Count  Dtype         
---  ------                                                                --------------  -----         
 0   SampleId                                                              96603 non-null  float64       
 1   Longitude                                                             96603 non-null  float64       
 2   Latitude                                                              96603 non-null  float64       
 3   ACTIVE CHLOROPHYLL-A ug/l                                             35770 non-null  float64       
 4   AIR TEMPERATURE DEG deg c                                             15657 non-null  float64       
 5   AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l                         41525 non-null  float64       
 6   DISSOLVED OXYGEN IN MG/L mg/l       

In [24]:
df_pivot3 = df_pivot3.rename_axis(None, axis=1)

In [25]:
df_pivot3.corr()['TOTAL NITROGEN mg/l']

SampleId                                                               -0.043412
Longitude                                                               0.056085
Latitude                                                                0.178259
ACTIVE CHLOROPHYLL-A ug/l                                               0.136843
AIR TEMPERATURE DEG deg c                                              -0.005245
AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l                           0.069650
DISSOLVED OXYGEN IN MG/L mg/l                                           0.046745
E.COLI BACTERIA mpn                                                     0.077122
NITRATE NITROGEN AS N (FILTERED SAMPLE) mg/l                            0.917744
ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE) mg/l                   0.382497
PH CORRECTED FOR TEMPERATURE (25 DEG C) su                             -0.009185
SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU) ppt   -0.556799
SECCHI DEPTH m              

In [26]:
df_pivot4 = df_pivot3.drop(columns = 'SampleId')
df_pivot4 = df_pivot4.set_index('DateTime')

In [27]:
#df_pivot3.to_csv('chemical.csv')

In [28]:
#preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold

#metrics
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error

#algorithms 
import xgboost as xgb

#hyptertuning
from sklearn.model_selection import RandomizedSearchCV

#Scalers 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [29]:
df_pivot4['TOTAL NITROGEN mg/l'] = df_pivot4['TOTAL NITROGEN mg/l'].replace(np.nan, 'nan')
df_nitro = df_pivot4[df_pivot4['TOTAL NITROGEN mg/l'] != 'nan']

In [30]:
df_nitro.info()
#drop all columns
#air temperature 
#specific conductivity
#tubidity tube 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44230 entries, 2017-01-16 14:15:00 to 2019-09-09 11:20:00
Data columns (total 23 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   Longitude                                                             44230 non-null  float64
 1   Latitude                                                              44230 non-null  float64
 2   ACTIVE CHLOROPHYLL-A ug/l                                             24623 non-null  float64
 3   AIR TEMPERATURE DEG deg c                                             471 non-null    float64
 4   AMMONIUM NITROGEN AS N (FILTERED SAMPLE) mg/l                         36237 non-null  float64
 5   DISSOLVED OXYGEN IN MG/L mg/l                                         43129 non-null  float64
 6   E.COLI BACTERIA mpn                                        

In [31]:
df_nitro = df_nitro.drop(columns = ['AIR TEMPERATURE DEG deg c','SPECIFIC CONDUCTIVITY us/cm', 'TURBIDITY TUBE cm'])

In [32]:
#Outliers
df_nitro = df_nitro[df_nitro['TOTAL NITROGEN mg/l'] < 50]

In [47]:
df_nitro.to_csv('dfnitro.csv')

In [33]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(df_nitro.drop(labels = 'TOTAL NITROGEN mg/l', axis = 1), 
                                                    df_nitro['TOTAL NITROGEN mg/l'], test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape

((30940, 19), (13260, 19))

In [34]:
xgb = xgb.XGBRegressor(random_state=0, n_jobs = -1)

In [35]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method=None, validate_parameters=False, verbosity=None)

In [36]:
y_pred = xgb.predict(X_test)

In [37]:
#Standard Scaler
print(r2_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

0.8711003126026277
0.8711179515393547
0.6759750132010013


In [38]:
features = X_train.columns
#get feature importance attributed by random forest model
importance = pd.concat([pd.Series(features),
                       pd.Series(xgb.feature_importances_)], axis = 1)

importance.columns = ['feature', 'importance']
importance.sort_values(by = 'importance', ascending = False)

,feature,importance
6,NITRATE NITROGEN AS N (FILTERED SAMPLE) mg/l,0.316955
1,Latitude,0.166834
15,TOTAL PHOSPHORUS mg/l,0.137878
11,TOTAL ALKALINITY AS CACO3 mg/l,0.078170
0,Longitude,0.056119
7,ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPL...,0.042419
18,WATER TEMPERATURE DEG deg c,0.029504
9,SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL S...,0.028356
14,TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE) mg/l,0.023000
17,TURBIDITY; NEPHELOMETRIC METHOD ntu,0.019999


In [39]:
X_train2 = X_train.drop(columns = ['DISSOLVED OXYGEN IN MG/L mg/l', 'E.COLI BACTERIA mpn', 'TOTAL DEPTH m',
       'TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C mg/l'])

X_test2 = X_test.drop(columns = ['DISSOLVED OXYGEN IN MG/L mg/l', 'E.COLI BACTERIA mpn', 'TOTAL DEPTH m',
       'TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C mg/l'])

In [40]:
xgb.fit(X_train2, y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method=None, validate_parameters=False, verbosity=None)

In [41]:
y_pred2 = xgb.predict(X_test2)

In [42]:
#Standard Scaler
print(r2_score(y_test, y_pred2))
print(explained_variance_score(y_test, y_pred2))
print(np.sqrt(mean_squared_error(y_test, y_pred2)))

0.8645247750150883
0.8645480792416336
0.693002260665122


In [43]:
features = X_train2.columns
#get feature importance attributed by random forest model
importance = pd.concat([pd.Series(features),
                       pd.Series(xgb.feature_importances_)], axis = 1)

importance.columns = ['feature', 'importance']
importance.sort_values(by = 'importance', ascending = False)

,feature,importance
4,NITRATE NITROGEN AS N (FILTERED SAMPLE) mg/l,0.288252
1,Latitude,0.186589
11,TOTAL PHOSPHORUS mg/l,0.140260
9,TOTAL ALKALINITY AS CACO3 mg/l,0.089225
0,Longitude,0.070221
5,ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPL...,0.046327
7,SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL S...,0.034332
14,WATER TEMPERATURE DEG deg c,0.025634
10,TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE) mg/l,0.024927
12,TOTAL SUSPENDED SOLIDS mg/l,0.021231


In [44]:
#save models and predictions 
predictions_df = pd.DataFrame()
predictions_df['Actual'] = y_test
predictions_df['pred1'] = y_pred
predictions_df['pred2'] = y_pred2

In [45]:
predictions_df.to_csv('predictions_df.csv')

In [46]:
import joblib
# save the model to disk
xgb = 'xgb.sav'
joblib.dump(xgb, xgb)

['xgb.sav']